In [1]:
import psycopg2
from dotenv import load_dotenv
load_dotenv()
import os

conn = psycopg2.connect(
    host=os.getenv('DB_HOST'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_NAME'),
    password=os.getenv('DB_PASSWORD')
)

cursor = conn.cursor()

In [64]:
def inserir_comando(sql, conn, **args):
    try:
        cursor.execute(sql, tuple(args.values()))
        conn.commit()
        print('Comando criado')
    except Exception as erro:
        conn.rollback()
        print(erro)

In [27]:
sql_criacao_tabela_usuario = rf''' 
    CREATE TABLE IF NOT EXISTS usuario (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(45) NOT NULL,
        login VARCHAR(45) NOT NULL,
        senha VARCHAR(45) NOT NULL,
        cargo VARCHAR(45) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [ ]:
inserir_comando(sql=sql_criacao_tabela_usuario, conn=conn)

Criado a tabela


In [24]:
sql_criacao_tabela_permissao = '''
    CREATE TABLE IF NOT EXISTS permissao (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(45) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [ ]:
inserir_comando(sql=sql_criacao_tabela_permissao, conn=conn)

In [37]:
sql_criacao_tabela_usuario_permissao = '''
    CREATE TABLE IF NOT EXISTS usuario_permissao (
        usuario_id INT NOT NULL,
        permissao_id INT NOT NULL,
        
        PRIMARY KEY (usuario_id, permissao_id),
        
        FOREIGN KEY (usuario_id) REFERENCES usuario(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (permissao_id) REFERENCES permissao(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [ ]:
inserir_comando(sql=sql_criacao_tabela_usuario_permissao, conn=conn)

Criado a tabela


In [58]:
sql_usuario = """
INSERT INTO usuario (nome, login, senha, cargo)
VALUES (%s, %s, %s, %s)
RETURNING id;
"""

cursor.execute(
    sql_usuario,
    ("Maria Silva", "maria", "hash_senha", "Analista")
)

conn.commit()

usuario_id = cursor.fetchone()[0]

InFailedSqlTransaction: ERRO:  transação atual foi interrompida, comandos ignorados até o fim do bloco de transação


In [54]:
sql_alterando_tabela = '''
    ALTER TABLE permissao
    ADD CONSTRAINT permissao_nome_unique UNIQUE (nome);
'''

In [ ]:
inserir_comando(sql=sql_alterando_tabela, conn=conn)

Criado a tabela


In [56]:
try:
    sql_permissao = """
    INSERT INTO permissao (nome)
    VALUES (%s)
    ON CONFLICT (nome) DO NOTHING
    RETURNING id;
    """

    cursor.execute(sql_permissao, ("ADMIN",))

    
    result = cursor.fetchone()
    
    conn.commit()

    if result:
        permissao_id = result[0]
    else:
        cursor.execute(
            "SELECT id FROM permissao WHERE nome = %s;",
            ("ADMIN",)
        )
        permissao_id = cursor.fetchone()[0]
except Exception as erro:
    conn.rollback()

In [63]:
#vinculos bitch
sql_usuario_permissao = """
INSERT INTO usuario_permissao (usuario_id, permissao_id)
VALUES (%s, %s)
ON CONFLICT DO NOTHING;
"""



inserir_comando(sql=sql_usuario_permissao, conn=conn, usuario_id=4, permissao_id=2)






Criado a tabela
